# 预处理

### 导入库

In [1]:
import pandas as pd
import numpy as np
import openpyxl
import warnings
warnings.filterwarnings("ignore")

In [2]:
import statsmodels
from statsmodels.stats.stattools import robust_kurtosis
import statsmodels.tsa.api as smt
#导入熵,最大峰值,和零跨率
from scipy.stats import entropy
from sklearn.preprocessing import MinMaxScaler

### 导入文件

In [3]:
# 输入文件: train_raw.xlsx, test_raw.xlsx
# 输出文件: 序列文件 特征文件

# 文件夹名称
dirName = './01-28/'

BOTTOMFLOOR = 1
TOPFLOOR = 35
# # 训练集文件地址
# # 训练集原始文件，导出序列文件，导出特征文件
doc_file = dirName + 'train_raw.xlsx'
seqName = dirName + 'train_sequence'
outputName  =dirName + 'train_output.xlsx'

# 测试集文件地址
# 测试集原始文件，导出序列文件，导出特征文件

# # 竹韵数据
BOTTOMFLOOR = 1
TOPFLOOR = 35
doc_file = dirName + 'test_raw.xlsx'
# seqName = dirName + 'test_sequence'
# outputName = dirName + 'test_output.xlsx'

seqName = dirName + 'test_sequence'
outputName = dirName + 'test_output.xlsx'

# 新时达数据
# BOTTOMFLOOR = 1
# TOPFLOOR = 4
# doc_file = dirName + '新时达数据.xlsx'
# seqName = dirName + 'test_sequence'
# outputName = dirName + 'test_output.xlsx'

# seqName = dirName + 'train_sequence'
# outputName  =dirName + 'train_output.xlsx'

# # 广州红树湾
# BOTTOMFLOOR = 1
# TOPFLOOR = 3
# doc_file = dirName + '广州红树湾.xlsx'
# seqName = dirName + 'test_sequence'
# outputName = dirName + 'test_output.xlsx'

# 广日默纳克
# BOTTOMFLOOR = 1
# TOPFLOOR = 4
# doc_file = dirName + '广日默纳克.xlsx'
# seqName = dirName + 'test_sequence'
# outputName = dirName + 'test_output.xlsx'

# # 西城花园
# BOTTOMFLOOR = 1
# TOPFLOOR = 7
# doc_file = dirName + '西城花园.xlsx'
# seqName = dirName + 'test_sequence'
# outputName = dirName + 'test_output.xlsx'

# # 广日新时达
# BOTTOMFLOOR = 1
# TOPFLOOR = 23
# doc_file = dirName + '广日新时达.xlsx'
# seqName = dirName + 'test_sequence'
# outputName = dirName + 'test_output.xlsx'

# 实验室默纳克
# BOTTOMFLOOR = 1
# TOPFLOOR = 4
# doc_file = dirName + '实验室默纳克.xlsx'
# seqName = dirName + 'test_sequence'
# outputName = dirName + 'test_output.xlsx'


# 读取原始文件的所有sheet信息
excel = openpyxl.load_workbook(doc_file)
# 文件的sheet名字
sheet_name = excel.sheetnames
# 文件的sheet数
sheet_num = len(sheet_name)
# 储存每个sheet的数据
sheet_list =[]
print(doc_file)

./01-28/test_raw.xlsx


### 筛选信息列(D0~D8)

In [4]:
# 读取文件
data_raw = pd.read_excel(doc_file,sheet_name=sheet_name,dtype='str')
# 筛选的列名
col_name = ['D0','D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8']
for name in sheet_name:
    # 筛选出的表格重新储存到data_raw
    data_raw[name] = data_raw[name][col_name]
    # 将原本的 COB-ID 或 ID 更换为 D0
    data_raw[name].rename(columns = {'ID':'D0'},inplace = True)
    data_raw[name].rename(columns = {'COB-ID':'D0'},inplace = True)

## 调整数据结构

### 提取序列

#### 所需函数
1. hexToDec 进制转换 16->10
2. filterID 筛选ID组合 [筛选列,数据列]
3. parseID  构造新ID和序列Sequence

In [5]:
# hexToDec 字节从16进制转到10进制
def hexToDec(sheet):
    # 表格中缺省值填充为 -1
    sheet.fillna('-1',inplace = True)
    # 对每一列进行进制转换
    for col in sheet.columns:
        sheet[col] = [int(item,16) for item in sheet[col]]
    return sheet

# filterID 筛选ID组合
def filterID(columns,startIdx = 0,endIdx = None):
    '''
    输入: [D0,D1,D2,....D8]
    输出: [[筛选列,数据列]...]
    例如：[
            [[D0],D1],
            ...
            [[D0,D1],D2],
            ...
            [[D4],D7],
            ...
        ]
    startIdx: 从第几项开始筛选，默认第0项
    endIdx:只取 [startIdx ~ i]列作为开头第一列筛选 
    [D0..] [D1..]... [Di..] 
    [Di+1..] 开始不再输出
    默认将所有列都筛选
    '''
    if(endIdx == None): 
        print("默认筛选所有列")
        endIdx = len(columns-1)
    # columns 格式转换：Index->list
    columns = list(columns)
    out = []
    # preCols 筛选列
    # dataIdx 数据列序号
    def parse(preCols,dataIdx):
        # 特定列为第一项筛选时结束
        if(len(preCols)>0 and columns.index(preCols[0])>=endIdx):return
        # 选数据列
        for col in columns[dataIdx:]:
            if(len(preCols)): out.append([preCols,col])
            # 数据列加入筛选列，进行下一次筛选
            parse(preCols+[col],columns.index(col)+1)
    parse([],startIdx)
    return out

# parseID
def parseID(sheet):
    # 返回 新ID和序列的dataframe
    sheet_df = pd.DataFrame()
    # 储存新ID与序列
    newIDs = []
    sequences =[] 
    # 进制转换
    sheet = hexToDec(sheet)
    # 筛选ID组合
    filterCols = filterID(sheet.columns,endIdx=4)

    #根据筛选ID组合筛选内容
    for filterCol,dataCol in filterCols:
        # rname: filterCol 筛选列中的数据
        # group: 在 rname 筛选下的表格
        for rname,group in sheet.groupby(filterCol):
            # 统一rname为列表
            if(type(rname)==int):rname = [rname]
            else:rname = list(rname)
            # 构造新ID和保存序列
            newID = ''.join([preItem + str(dataItem) + '_' for preItem,dataItem in zip(filterCol,rname)])+dataCol
            sequence = list(group[dataCol])
            newIDs.append(newID)
            sequences.append(sequence)
    
    sheet_df['ID'] = newIDs
    sheet_df['Sequence'] = sequences
    # sheet_df.set_index('ID',inplace=True)
    return sheet_df

### 组装数据

#### 单列

In [6]:
data = pd.DataFrame()
for name in sheet_name:
    data = pd.concat([data,parseID(data_raw[name])])
# 拼接后重置index，使之保持正确
data.reset_index(drop = True,inplace=True)

#### 差分序列

In [7]:
differences = []
for sequence in data['Sequence']:
    differences.append(np.diff(sequence))
data['Difference'] = differences
print('添加差分后data形状：',data.shape)

添加差分后data形状： (201524, 3)


### 删除冗余数据

#### 无意义数据
1. 重复数据
2. 序列长度小于等于2
3. 序列方差小于0.1

In [8]:
drop_index = []
for i,sequence in enumerate(data['Difference']):
    if(len(set(sequence))<1):
        drop_index.append(i)
for i,sequence in enumerate(data['Sequence']):
    if(len(set(sequence))==1
    or len(sequence)<=2
    or np.var(np.array(sequence))<0.1):
        drop_index.append(i)
# 删除特定行
data.drop(drop_index,inplace = True)
# 保持index顺序正确
data.reset_index(drop = True,inplace=True)
print(f"删除不变数据{len(set(drop_index))}条。")
print('删除不变数据后data形状：',data.shape)

删除不变数据192136条。
删除不变数据后data形状： (9388, 3)


#### 循环数据
使用算法：
https://writings.sh/post/algorithm-repeated-string-pattern

In [9]:
cycle_index = []
for i,sequence in enumerate(data['Sequence']):
    flag = 0
    doubleSeq = sequence[1:]+sequence[:-1]
    for idx in range(0,len(doubleSeq)-len(sequence)):
        if(sequence == doubleSeq[idx:idx+len(sequence)]):
            flag = 1
            break
    if(flag): cycle_index.append(i)
# 删除特定行
data.drop(cycle_index,inplace = True)
# 保持index顺序正确
data.reset_index(drop = True,inplace=True)
print(f"删除循环数据{len(cycle_index)}条。")
print('删除循环数据后data形状：',data.shape)

删除循环数据306条。
删除循环数据后data形状： (9082, 3)


## 特征工程


### 序列特征

序列重复项去重

In [10]:
depublication = []
for sequence in data['Sequence']:
    tmpSeq = [sequence[i] for i in range(len(sequence))
     if sequence[i-1]!=sequence[i] or i==0]
    depublication.append(tmpSeq)
data['Depublication'] = depublication

### 范围归一化

将楼层范围信号值归到[0,1]

楼层范围以下信号<0，楼层范围以上信号>1

为避免出现负数，归一化对数据平移，实际楼层信号标准化到[1,2]

In [12]:
maxSignal = (255 - BOTTOMFLOOR)/(TOPFLOOR-BOTTOMFLOOR)
minSignal = (0 - BOTTOMFLOOR)/(TOPFLOOR-BOTTOMFLOOR)
print(maxSignal,minSignal)
# 范围归一的dataframe
data_nomolized = data.copy(deep=True)
# 将两列范围归一化
for col in ['Sequence','Depublication']:
    for index,sequence in enumerate(data_nomolized[col]):
        data_nomolized[col][index] = [(x - BOTTOMFLOOR)/(TOPFLOOR-BOTTOMFLOOR)+1 for x in sequence]

7.470588235294118 -0.029411764705882353


### 分箱

3个箱：小于楼层范围占比，楼层范围占比，大于楼层占比

In [15]:
bins = np.array([minSignal-1,-0.001,1,maxSignal])+1
sequence_bins = pd.DataFrame()
for i in range(data_nomolized.shape[0]):
    index = data_nomolized['Sequence'][i]
    sequence_bin = pd.Series(index).value_counts(bins=bins,sort=False)
    sequence_bins = sequence_bins.append(pd.Series(sequence_bin.values)/len(index),ignore_index=True)
# 重命名列名
sequence_bins.columns = [f'bin_{i}' for i in range(len(bins)-1)]

### 数字特征

1. 最大值

2. 最小值

3. 最左值

4. 最右值

5. 左小 判断最小值是否为最左值

6. 右大 判断最大值是否为最右值

7. 平均值

8. 最小二乘回归斜率

9. 最小二乘回归截距

10. 方差

11. 曲率

12. 求和

13. 下4分位数，中位数，上4分位数

14. 序列长与楼层范围占比


In [18]:
data_feature = pd.DataFrame()
data_feature['ID'] = data['ID']
# for col in ['Sequence','Depublication']:
for col in ['Depublication']:
    # 最大值
    maxs = []
    # 最小值
    mins = []
    # 最左值
    lefts = []
    # 最右值
    rights = []
    # 左小
    leftmin = []
    # 右大
    rightmax = []
    # 平均值
    averages = []
    # 最小二乘回归斜率
    slopes = []
    # 最小二乘回归截距
    intercepts = []
    # 方差
    vars = []
    # 曲率
    curvatures = []
    # 求和
    sums = []
    # 下4分位数，中位数，上4分位数
    aa = []
    bb = []
    cc = []
    # 序列长与楼层范围占比
    lengthSlope = []
    
    for i,index in enumerate(data_nomolized[col]):
        # 计算斜率与截距
        slope, intercept = np.polyfit(range(len(index)), index, 1)
        diffsum = np.sum(np.diff(index,1))
        diff = np.diff(index,1)
        a,b,c = np.percentile(diff, (25, 50, 75), interpolation='midpoint')
        
        maxs.append(np.max(index))
        mins.append(np.min(index))
        lefts.append(index[0])
        rights.append(index[-1])
        if(maxs[-1] == rights[-1]):
            rightmax.append(1)
        else:
            rightmax.append(0)
        if(mins[-1] == lefts[-1]):
            leftmin.append(1)
        else:
            leftmin.append(0)
        averages.append(np.average(index))
        slopes.append(slope)
        intercepts.append(intercept)
        vars.append(np.var(index))
        aa.append(a)
        bb.append(b)
        cc.append(c)
        curvatures.append(np.average(np.diff(index,1)))
        lengthSlope.append(len(index)/len(range(BOTTOMFLOOR,TOPFLOOR)))
    data_feature[col[:3]+'-max'] = maxs
    data_feature[col[:3]+'-min'] = mins
    data_feature[col[:3]+'-left'] = lefts
    data_feature[col[:3]+'-right'] = rights
    data_feature[col[:3]+'-leftmin'] = leftmin
    data_feature[col[:3]+'-rightmax'] = rightmax
    data_feature[col[:3]+'-average'] = averages
    data_feature[col[:3]+'-a'] = aa
    data_feature[col[:3]+'-b'] = bb
    data_feature[col[:3]+'-c'] = cc
    data_feature[col[:3]+'-slope'] = slopes
    data_feature[col[:3]+'-intercept'] = intercepts
    data_feature[col[:3]+'-vars'] = vars
    data_feature[col[:3]+'-curvature'] = curvatures
    data_feature[col[:3]+'-lenSlope'] = lengthSlope

In [19]:
# 加上分箱结果列
data_feature = pd.concat([data_feature,sequence_bins],axis=1)

## 设置训练集label

In [20]:
# 楼层位置
# 竹韵标签
floorID = ['D01856_D129_D20_D371_D5','D01856_D129_D371_D5','D01856_D20_D371_D5',
       'D01856_D371_D5','D01856_D129_D20_D371_D5','D01856_D129_D371_D5',
       'D01856_D20_D371_D5','D01856_D371_D5','D01856_D129_D20_D371_D5',
       'D01856_D129_D371_D5','D01856_D20_D371_D5','D01856_D371_D5',
       'D01856_D129_D20_D371_D5','D01856_D129_D371_D5','D01856_D20_D371_D5',
       'D01856_D371_D5']
# 新时达标签
# floorID = ["D08_D10_D21_D3","D08_D10_D21_D42_D50_D6","D08_D10_D21_D42_D6","D08_D10_D21_D50_D6","D08_D10_D21_D6","D08_D10_D2134_D30_D40_D50_D6","D08_D10_D2134_D30_D40_D6","D08_D10_D2134_D30_D50_D6","D08_D10_D2134_D30_D6","D08_D10_D2134_D40_D50_D6","D08_D10_D2134_D40_D6","D08_D10_D2134_D50_D6","D08_D10_D2134_D6","D08_D10_D2188_D3","D08_D10_D42_D50_D6","D08_D10_D42_D6","D08_D14_D210_D34_D40_D50_D61_D7","D08_D14_D210_D34_D40_D50_D7","D08_D14_D210_D34_D40_D61_D7","D08_D14_D210_D34_D40_D7","D08_D14_D210_D34_D50_D61_D7","D08_D14_D210_D34_D50_D7","D08_D14_D210_D34_D61_D7","D08_D14_D210_D34_D7","D08_D14_D210_D40_D50_D61_D7","D08_D14_D210_D40_D50_D7","D08_D14_D210_D40_D61_D7","D08_D14_D210_D40_D7","D08_D14_D210_D50_D61_D7","D08_D14_D210_D50_D7","D08_D14_D210_D61_D7","D08_D14_D210_D7","D08_D14_D34_D40_D50_D61_D7","D08_D14_D34_D40_D50_D7","D08_D14_D34_D40_D61_D7","D08_D14_D34_D40_D7","D08_D14_D34_D50_D61_D7","D08_D14_D34_D50_D7","D08_D14_D34_D61_D7","D08_D14_D34_D7","D08_D14_D40_D50_D61_D7","D08_D14_D40_D61_D7","D08_D14_D50_D61_D7","D08_D14_D61_D7","D08_D21_D3","D08_D21_D42_D50_D6","D08_D21_D42_D6","D08_D21_D50_D6","D08_D21_D6","D08_D210_D34_D40_D50_D61_D7","D08_D210_D34_D40_D50_D7","D08_D210_D34_D40_D61_D7","D08_D210_D34_D40_D7","D08_D210_D34_D50_D61_D7","D08_D210_D34_D50_D7","D08_D210_D34_D61_D7","D08_D210_D34_D7","D08_D210_D40_D50_D61_D7","D08_D210_D40_D61_D7","D08_D210_D50_D61_D7","D08_D210_D61_D7","D08_D2134_D30_D40_D50_D6","D08_D2134_D30_D40_D6","D08_D2134_D30_D50_D6","D08_D2134_D30_D6","D08_D2134_D40_D50_D6","D08_D2134_D40_D6","D08_D2134_D50_D6","D08_D2134_D6","D08_D2188_D3","D08_D34_D40_D50_D61_D7","D08_D34_D40_D61_D7","D08_D34_D50_D61_D7","D08_D34_D61_D7","D08_D42_D50_D6","D08_D42_D6","D10_D21_D3","D10_D21_D42_D50_D6","D10_D21_D42_D6","D10_D21_D50_D6","D10_D21_D6","D10_D2134_D30_D40_D50_D6","D10_D2134_D30_D40_D6","D10_D2134_D30_D50_D6","D10_D2134_D30_D6","D10_D2134_D40_D50_D6","D10_D2134_D40_D6","D10_D2134_D50_D6","D10_D2134_D6","D10_D2171_D3","D10_D2188_D3","D10_D42_D50_D6","D10_D42_D6","D14_D210_D34_D40_D50_D61_D7","D14_D210_D34_D40_D50_D7","D14_D210_D34_D40_D61_D7","D14_D210_D34_D40_D7","D14_D210_D34_D50_D61_D7","D14_D210_D34_D50_D7","D14_D210_D34_D61_D7","D14_D210_D34_D7","D14_D210_D40_D50_D61_D7","D14_D210_D40_D50_D7","D14_D210_D40_D61_D7","D14_D210_D40_D7","D14_D210_D50_D61_D7","D14_D210_D50_D7","D14_D210_D61_D7","D14_D210_D7","D14_D34_D40_D50_D61_D7","D14_D34_D40_D50_D7","D14_D34_D40_D61_D7","D14_D34_D40_D7","D14_D34_D50_D61_D7","D14_D34_D50_D7","D14_D34_D61_D7","D14_D34_D7","D14_D40_D50_D61_D7","D14_D40_D61_D7","D14_D50_D61_D7","D14_D61_D7","D21_D3","D21_D42_D50_D6","D21_D42_D6","D21_D50_D6","D21_D6","D210_D34_D40_D50_D61_D7","D210_D34_D40_D50_D7","D210_D34_D40_D61_D7","D210_D34_D40_D7","D210_D34_D50_D61_D7","D210_D34_D50_D7","D210_D34_D61_D7","D210_D34_D7","D210_D40_D50_D61_D7","D210_D40_D61_D7","D210_D50_D61_D7","D210_D61_D7","D2134_D30_D40_D50_D6","D2134_D30_D40_D6","D2134_D30_D50_D6","D2134_D30_D6","D2134_D40_D50_D6","D2134_D40_D6","D2134_D50_D6","D2134_D6","D2171_D3","D2188_D3","D34_D40_D50_D61_D7","D34_D40_D61_D7","D34_D50_D61_D7","D34_D61_D7"]


data_feature['Label'] = np.zeros((data_feature.shape[0],1))
data_feature['Label'][data_feature.loc[data_feature['ID'].isin(floorID)].index] = 1
# data_feature['Label'][data_feature.loc[data_feature['ID'].isin(updownID)].index] = 2
# data_feature['Label'][data_feature.loc[data_feature['ID'].isin(doorID)].index] = 3
# data_feature['Label'][data_feature.loc[data_feature['ID'].isin(openID)].index] = 4

## 特征处理结束

In [21]:
# 加上bins列
data = pd.concat([data,sequence_bins],axis=1)
data['Label'] = data_feature['Label']
data.to_pickle(seqName+'.pkl')
data.to_excel(seqName+'.xlsx')

In [22]:
data_feature.to_excel(outputName)